<a href="https://colab.research.google.com/github/Valerie6048/latihan_github/blob/main/Recommender_System_using_Cosine_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Membaca data dari file CSV
df = pd.read_csv('/content/drive/MyDrive/For Capstone/Collecting data/Place Detail (Scored + Keyword 1 & 2 Extracted  + Additional Feature (longlang, contact etc)) + (finished Vectorized).csv')

In [ ]:
# Mengganti nilai np.nan dengan string kosong
df['One_Keywords'] = df['One_Keywords'].fillna("")

# Inisialisasi objek TfidfVectorizer dan lakukan vektorisasi
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['One_Keywords'])

# Menghitung cosine similarity antara representasi vektor TF-IDF
cosine_sim = cosine_similarity(X)

In [ ]:
def get_recommendations(item_title, cosine_similarities, df, top_n=5):
    item_index = df[df['Name'] == item_title].index[0]  # Get the index of the reference item
    similarity_scores = list(enumerate(cosine_similarities[item_index]))  # Get the similarity scores with other items
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)  # Sort by similarity scores
    top_items = similarity_scores[1:top_n+1]  # Take the top N items (excluding the reference item)
    top_item_indices = [i[0] for i in top_items]  # Get the indices of the top items
    top_item_data = df.iloc[top_item_indices].reset_index(drop=True)  # Get the data of the top items and reset index

    # Create a list of dictionaries for the recommended items
    recommendations = []
    for _, row in top_item_data.iterrows():
        recommendation = {
            'name': row['Name'],
            'address': row['Formatted Address'],
            'rating': row['rating'],
            'total_review': row['total_reviews'],
            'url_photo': row.get('Photo URL', 'N/A'),
            'place id': row.get('Place ID'),
            'place url': row.get('Place URL'),
            'contact': row.get('Contact'),
            'longitude': row.get('Longitude'),
            'latitude': row.get('Latitude')

        }
        recommendations.append(recommendation)
    return recommendations


In [ ]:
def filter_data(data, city_name):
  filtered_data = []
  for item in data:
    if city_name in item["address"]:
      filtered_data.append(item)
  return filtered_data

In [ ]:
nama_caffe = input()

In [ ]:
nama_kota = input()

In [ ]:
# Get recommendations
recommendations = get_recommendations(nama_caffe, cosine_sim, df, top_n=20)

# Print the recommendations
print("Rekomendasi:")
for recommendation in recommendations:
    print(recommendation)

In [ ]:
filtered_data = filter_data(recommendations, nama_kota)

print(json.dumps(filtered_data, indent=4))